In [1]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import import_ipynb


pd.set_option('display.max_columns', None)

In [3]:
raw_19 = pd.read_csv('../data/PitchData_19.csv')
raw_20 = pd.read_csv('../data/PitchData_20.csv')
raw_21 = pd.read_csv('../data/PitchData_21.csv')

In [4]:
raw_all = pd.concat([raw_21, raw_20, raw_19], ignore_index=True)
data = raw_all.copy()
data.drop(columns=data.columns[0], inplace=True)

In [5]:
# data = raw_21.copy()

In [6]:
data = data[data['game_type']=='R'].copy()

In [7]:
data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
10786,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,0.69,1.38,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,NaN,NaN,518595.0,NaN,NaN,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,0,5,0,5,Infield shift,Strategic,148.0,0.000,-0.073
10787,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,NaN,NaN,NaN,NaN,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-0.77,0.48,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,0,5,0,5,Infield shift,Strategic,315.0,0.000,-0.027
10788,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,NaN,NaN,NaN,NaN,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-0.65,-0.51,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,0,5,0,5,Infield shift,Standard,328.0,0.000,-0.020
10789,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,NaN,ball,NaN,NaN,NaN,NaN,12.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,B,NaN,NaN,0,0,2021,-0.69,-0.69,2.10,3.89,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,2.375830,-109.205830,2.277617,-5.902656,19.427562,-38.284747,3.31,1.48,NaN,NaN,NaN,74.5,2017.0,5.9,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,Curveball,5,0,0,5,0,5,0,5,Standard,Standard,330.0,0.000,0.016
10790,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,"Brandon Nimmo grounds out, second baseman Ozzi...",R,L,L,ATL,NYM,X,4.0,ground_ball,1,0,2021,0.63,1.28,-0.31,2.80,NaN,NaN,NaN,1,9,Top,144.83,145.72,NaN,NaN,518595.0,NaN,NaN,-5.868477,-132.500539,-6.486796,8.700586,30.117690,-15.941174,3.49,1.60,9.0,93.3,-18.0,90.9,2281.0,6.3,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.15,0.100,0.090,0.0,1.0,0.0,0.0,2.0,60,2,4-Seam Fastball,5,0,0,5,0,5,0,5,Standard,Standard,143.0,0.002,-0.189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# RE24

In [8]:
# replace onbase info with 0 and 1s

data.loc[data['on_3b'].notna(), 'on_3b'] = 1
data.loc[data['on_2b'].notna(), 'on_2b'] = 1
data.loc[data['on_1b'].notna(), 'on_1b'] = 1
data[['on_3b', 'on_2b', 'on_1b']] = data[['on_3b', 'on_2b', 'on_1b']].fillna(0)

In [9]:
data_pa = data.groupby(['game_date', 'game_pk', 'at_bat_number']).agg({'game_year':max,
                                                                       'home_team':max,
                                                                       'away_team':max,
                                                                       'player_name':max,
                                                                       'batter':max,
                                                                       'pitcher':max,
                                                                        'inning_topbot':max,
                                                                        'inning':max,
                                                                        'outs_when_up':max,
                                                                        'home_score':'last',
                                                                        'away_score':'last',
                                                                        'post_home_score':'first',
                                                                        'post_away_score':'first',
                                                                        'bat_score':'last',
                                                                        'post_bat_score':'first',
                                                                        'on_1b':'last',
                                                                        'on_2b':'last',
                                                                        'on_3b':'last',
                                                                        'events':'first',
                                                                        'delta_home_win_exp':sum,
                                                                        'delta_run_exp':sum}).copy()
data_pa = data_pa.reset_index()

In [10]:
data_pa['PA_score'] = data_pa['post_bat_score'] - data_pa['bat_score']

In [11]:
data_inning = data_pa.groupby(['game_date', 'game_pk', 'home_team', 
                               'away_team', 'inning_topbot', 'inning']).agg({'post_bat_score':max})
data_inning = data_inning.reset_index()

In [12]:
# group again to get final score of inning
data_inning.rename(columns={'post_bat_score':'inning_score'}, inplace=True)

In [13]:
# join inning score with data_pa
data_pa = data_pa.merge(data_inning, on=['game_date', 'game_pk', 'home_team', 
                                         'away_team', 'inning_topbot', 'inning'], how='outer')

In [14]:
data_inning

,game_date,game_pk,home_team,away_team,inning_topbot,inning,inning_score
0,2019-03-20,566083,OAK,SEA,Bot,1,1
1,2019-03-20,566083,OAK,SEA,Bot,2,2
2,2019-03-20,566083,OAK,SEA,Bot,3,4
3,2019-03-20,566083,OAK,SEA,Bot,4,4
4,2019-03-20,566083,OAK,SEA,Bot,5,4
...,...,...,...,...,...,...,...
101822,2021-10-03,632254,ATL,NYM,Top,5,0
101823,2021-10-03,632254,ATL,NYM,Top,6,0
101824,2021-10-03,632254,ATL,NYM,Top,7,0
101825,2021-10-03,632254,ATL,NYM,Top,8,0


In [15]:
data_pa.shape

(434372, 26)

In [16]:
# calculate score difference from start of PA and end of inning
data_pa['inning_score_diff'] = data_pa['inning_score'] - data_pa['bat_score']

In [17]:
data_pa

,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff
0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0
1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0
2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0
3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0
4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434367,2021-10-03,632254,57,2021,ATL,NYM,"Gsellman, Robert",663586,607229,Bot,8,1,5,0,5,0,5,5,0.0,0.0,0.0,strikeout,0.000,-0.163,0,5,0
434368,2021-10-03,632254,58,2021,ATL,NYM,"Gsellman, Robert",594807,607229,Bot,8,2,5,0,5,0,5,5,0.0,0.0,0.0,field_out,0.000,-0.103,0,5,0
434369,2021-10-03,632254,59,2021,ATL,NYM,"Smith, Will",641645,519293,Top,9,0,5,0,5,0,0,0,0.0,0.0,0.0,strikeout,0.003,-0.242,0,0,0
434370,2021-10-03,632254,60,2021,ATL,NYM,"Smith, Will",607043,519293,Top,9,1,5,0,5,0,0,0,0.0,0.0,0.0,field_out,0.002,-0.163,0,0,0


In [18]:
# create base_out state label
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==0)&(data_pa['on_3b']==0), 'on_base'] = 0
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==0)&(data_pa['on_3b']==0), 'on_base'] = 1
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==1)&(data_pa['on_3b']==0), 'on_base'] = 2
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==0)&(data_pa['on_3b']==1), 'on_base'] = 3
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==1)&(data_pa['on_3b']==0), 'on_base'] = 12
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==0)&(data_pa['on_3b']==1), 'on_base'] = 13
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==1)&(data_pa['on_3b']==1), 'on_base'] = 23
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==1)&(data_pa['on_3b']==1), 'on_base'] = 123

In [19]:
# # drop 9th innings
# data_pa_drop = data_pa[data_pa['inning']!=9].copy()

In [20]:
re24 = data_pa.groupby(['on_base', 'outs_when_up']).agg({'inning_score_diff':'mean'})
re24 = re24.reset_index().copy()

In [21]:
re24_long = re24.copy()
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==0), 'baseOutState'] = '0_0'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==0), 'baseOutState'] = '1_0'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==0), 'baseOutState'] = '2_0'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==0), 'baseOutState'] = '3_0'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==0), 'baseOutState'] = '12_0'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==0), 'baseOutState'] = '13_0'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==0), 'baseOutState'] = '23_0'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==0), 'baseOutState'] = '123_0'
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==1), 'baseOutState'] = '0_1'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==1), 'baseOutState'] = '1_1'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==1), 'baseOutState'] = '2_1'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==1), 'baseOutState'] = '3_1'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==1), 'baseOutState'] = '12_1'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==1), 'baseOutState'] = '13_1'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==1), 'baseOutState'] = '23_1'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==1), 'baseOutState'] = '123_1'
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==2), 'baseOutState'] = '0_2'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==2), 'baseOutState'] = '1_2'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==2), 'baseOutState'] = '2_2'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==2), 'baseOutState'] = '3_2'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==2), 'baseOutState'] = '12_2'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==2), 'baseOutState'] = '13_2'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==2), 'baseOutState'] = '23_2'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==2), 'baseOutState'] = '123_2'
# drop on_base and outs
re24_long.drop(['on_base', 'outs_when_up'], axis=1, inplace=True)
re24_long.rename(columns={'inning_score_diff':'run_expectancy'}, inplace=True)

### RE24 Long Format:

In [22]:
re24_long

,run_expectancy,baseOutState
0,0.525301,0_0
1,0.280440,0_1
2,0.107299,0_2
3,0.940411,1_0
4,0.556846,1_1
5,0.235307,1_2
6,1.148116,2_0
7,0.714286,2_1
8,0.333639,2_2
9,1.406993,3_0


### RE24 Wide:

In [23]:
re24 = re24.pivot(index='on_base', columns='outs_when_up', values='inning_score_diff').reset_index().copy()
re24

outs_when_up,on_base,0,1,2
0,0.0,0.525301,0.280440,0.107299
1,1.0,0.940411,0.556846,0.235307
2,2.0,1.148116,0.714286,0.333639
3,3.0,1.406993,0.982439,0.385291
4,12.0,1.556552,0.963288,0.456966
5,13.0,1.751656,1.211726,0.525179
6,23.0,2.032967,1.439815,0.603007
7,123.0,2.399430,1.657901,0.794617


# Turn RE charts into Dict for faster operation

In [25]:
# re24 Long:

re24_dict_long = {}
for i in range(24):
    re24_dict_long[re24_long.iloc[i]['baseOutState']] = re24_long.iloc[i]['run_expectancy']

In [26]:
# re24 wide:
re24[3] = 0
re24_dict = {}
for a in range(8):
    outs = {}
    re24_dict[re24.iloc[a]['on_base']] = outs
    for b in range(4):
        outs[b] = re24.iloc[a][b]

# Run Value

In [27]:
# get the max inning number of each game
max_inning = data_pa.groupby(['game_date', 'game_pk']).agg({'inning':max}).copy()
max_inning = max_inning.reset_index().copy()
max_inning.rename(columns={'inning':'max_inning'}, inplace=True)
# merge max_inning with data_pa
data_pa = data_pa.merge(max_inning, on=['game_date', 'game_pk'], how='outer').copy()

In [28]:
max_inning.head()

,game_date,game_pk,max_inning
0,2019-03-20,566083,9
1,2019-03-21,566084,12
2,2019-03-28,565220,9
3,2019-03-28,565702,9
4,2019-03-28,565799,9


In [29]:
# drop all bottom of last inning
data_pa_drop = data_pa[~((data_pa.inning_topbot=='Bot')&(data_pa.inning==data_pa.max_inning))].copy()
data_pa_drop = data_pa_drop.reset_index().copy()

In [30]:
# label rows based on whether it was the last out of the inning

data_pa_drop['is_last_out'] = 1
for index, row in data_pa_drop.iloc[:-1].iterrows():
    if data_pa_drop.loc[index, 'inning_topbot'] == data_pa_drop.loc[index+1, 'inning_topbot']:
        data_pa_drop.at[index, 'is_last_out'] = 0
    else:
        continue

In [31]:
# get the post outs and post on base info
data_pa_drop['post_outs'] = 3
data_pa_drop['post_1b'] = 0
data_pa_drop['post_2b'] = 0
data_pa_drop['post_3b'] = 0
for index, row in data_pa_drop.iloc[:-1].iterrows():
    if data_pa_drop.loc[index, 'is_last_out'] == 0:
        data_pa_drop.at[index, 'post_outs'] = data_pa_drop.loc[index+1, 'outs_when_up']
        data_pa_drop.at[index, 'post_1b'] = data_pa_drop.loc[index+1, 'on_1b']
        data_pa_drop.at[index, 'post_2b'] = data_pa_drop.loc[index+1, 'on_2b']
        data_pa_drop.at[index, 'post_3b'] = data_pa_drop.loc[index+1, 'on_3b']
    else:
        continue

In [32]:
data_pa_drop[data_pa_drop.events=='single']

,index,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,on_base,max_inning,is_last_out,post_outs,post_1b,post_2b,post_3b
2,2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0,0.0,9,0,2,1,0,0
11,11,2019-03-20,566083,12,2019,OAK,SEA,"Gonzales, Marco",640461,594835,Bot,2,0,1,0,1,0,1,1,0.0,0.0,0.0,single,0.031,0.408,0,2,1,0.0,9,0,0,1,0,0
14,14,2019-03-20,566083,15,2019,OAK,SEA,"Gonzales, Marco",543760,594835,Bot,2,1,1,0,2,0,1,2,0.0,1.0,1.0,single,0.063,0.916,1,2,1,23.0,9,0,1,1,0,1
16,16,2019-03-20,566083,17,2019,OAK,SEA,"Fiers, Mike",542921,571666,Top,3,0,2,0,2,0,0,0,0.0,0.0,0.0,single,-0.042,0.407,0,5,5,0.0,9,0,0,1,0,0
18,18,2019-03-20,566083,19,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,3,1,2,0,2,1,0,1,0.0,1.0,0.0,single,-0.087,0.648,1,5,5,2.0,9,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421798,434331,2021-10-03,632254,21,2021,ATL,NYM,"Williams, Trevor",518692,592866,Bot,3,1,2,0,2,0,2,2,1.0,0.0,0.0,single,0.043,0.475,0,5,3,1.0,9,0,1,1,0,1
421801,434334,2021-10-03,632254,24,2021,ATL,NYM,"Williams, Trevor",594807,592866,Bot,3,2,4,0,5,0,4,5,0.0,1.0,0.0,single,0.036,0.734,1,5,1,2.0,9,0,2,1,0,0
421807,434340,2021-10-03,632254,30,2021,ATL,NYM,"Williams, Trevor",518595,592866,Bot,4,0,5,0,5,0,5,5,0.0,0.0,0.0,single,0.006,0.403,0,5,0,0.0,9,0,0,1,0,0
421812,434345,2021-10-03,632254,35,2021,ATL,NYM,"Matzek, Tyler",542340,554431,Top,5,0,5,0,5,0,0,0,0.0,0.0,0.0,single,-0.020,0.403,0,0,0,0.0,9,0,0,1,0,0


In [33]:
data_pa_drop.rename(columns={'on_base':'pre_on_base'}, inplace=True)

In [34]:
# on base post
data_pa_drop.loc[(data_pa_drop['post_1b']==0)&(data_pa_drop['post_2b']==0)&(data_pa_drop['post_3b']==0), 'post_on_base'] = 0
data_pa_drop.loc[(data_pa_drop['post_1b']==1)&(data_pa_drop['post_2b']==0)&(data_pa_drop['post_3b']==0), 'post_on_base'] = 1
data_pa_drop.loc[(data_pa_drop['post_1b']==0)&(data_pa_drop['post_2b']==1)&(data_pa_drop['post_3b']==0), 'post_on_base'] = 2
data_pa_drop.loc[(data_pa_drop['post_1b']==0)&(data_pa_drop['post_2b']==0)&(data_pa_drop['post_3b']==1), 'post_on_base'] = 3
data_pa_drop.loc[(data_pa_drop['post_1b']==1)&(data_pa_drop['post_2b']==1)&(data_pa_drop['post_3b']==0), 'post_on_base'] = 12
data_pa_drop.loc[(data_pa_drop['post_1b']==1)&(data_pa_drop['post_2b']==0)&(data_pa_drop['post_3b']==1), 'post_on_base'] = 13
data_pa_drop.loc[(data_pa_drop['post_1b']==0)&(data_pa_drop['post_2b']==1)&(data_pa_drop['post_3b']==1), 'post_on_base'] = 23
data_pa_drop.loc[(data_pa_drop['post_1b']==1)&(data_pa_drop['post_2b']==1)&(data_pa_drop['post_3b']==1), 'post_on_base'] = 123

In [35]:
# add pre_base_out_state label
data_pa_drop.loc[(data_pa_drop['pre_on_base']==0)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '0_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==1)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '1_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==2)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '2_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==3)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '3_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==12)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '12_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==13)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '13_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==23)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '23_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==123)&(data_pa_drop['outs_when_up']==0), 'pre_baseOutState'] = '123_0'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==0)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '0_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==1)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '1_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==2)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '2_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==3)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '3_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==12)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '12_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==13)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '13_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==23)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '23_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==123)&(data_pa_drop['outs_when_up']==1), 'pre_baseOutState'] = '123_1'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==0)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '0_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==1)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '1_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==2)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '2_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==3)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '3_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==12)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '12_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==13)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '13_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==23)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '23_2'
data_pa_drop.loc[(data_pa_drop['pre_on_base']==123)&(data_pa_drop['outs_when_up']==2), 'pre_baseOutState'] = '123_2'

In [36]:
data_pa_drop.head()

,index,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,pre_on_base,max_inning,is_last_out,post_outs,post_1b,post_2b,post_3b,post_on_base,pre_baseOutState
0,0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0
1,1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1
2,2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0,0.0,9,0,2,1,0,0,1.0,0_2
3,3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0,1.0,9,1,3,0,0,0,0.0,1_2
4,4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1,0.0,9,0,1,0,0,0,0.0,0_0


In [37]:
# calculate Pre and Post RE, and Run Value
for index, row in data_pa_drop.iterrows():
    pre_on_base = row.pre_on_base
    pre_outs = row.outs_when_up
    
    post_on_base = row.post_on_base
    post_outs = row.post_outs
    
    data_pa_drop.at[index, 'pre_RE'] = re24_dict[pre_on_base][pre_outs]
    data_pa_drop.at[index, 'post_RE'] = re24_dict[post_on_base][post_outs]

data_pa_drop['post_RE'] = data_pa_drop['post_RE'] + data_pa_drop['PA_score']

In [38]:
data_pa_drop['run_value'] = data_pa_drop['post_RE'] - data_pa_drop['pre_RE']

In [39]:
data_pa_drop['events'].unique()

array(['field_out', 'single', 'strikeout', 'home_run', 'fielders_choice',
       'grounded_into_double_play', 'hit_by_pitch', 'walk', 'double',
       'sac_bunt', 'sac_fly', 'force_out', 'field_error', 'triple',
       'fielders_choice_out', 'double_play', 'caught_stealing_2b', None,
       'strikeout_double_play', 'sac_bunt_double_play',
       'caught_stealing_home', 'pickoff_1b', 'other_out',
       'catcher_interf', 'caught_stealing_3b', 'sac_fly_double_play',
       'pickoff_2b', 'pickoff_caught_stealing_home', 'passed_ball',
       'triple_play', 'pickoff_caught_stealing_3b', 'stolen_base_2b',
       'runner_double_play', 'pickoff_3b', 'game_advisory', 'wild_pitch',
       'pickoff_caught_stealing_2b'], dtype=object)

In [40]:
data_pa_drop['batter']

0         543829
1         571745
2         457803
3         429665
4         657656
           ...  
421834    663586
421835    594807
421836    641645
421837    607043
421838    596019
Name: batter, Length: 421839, dtype: int64

In [41]:
data_pa_drop[(data_pa_drop.batter==545361)&(data_pa_drop.game_year==2021)]['run_value'].mean()

0.09410244699194453

In [42]:
data_pa_drop

,index,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,pre_on_base,max_inning,is_last_out,post_outs,post_1b,post_2b,post_3b,post_on_base,pre_baseOutState,pre_RE,post_RE,run_value
0,0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
1,1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141
2,2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0,0.0,9,0,2,1,0,0,1.0,0_2,0.107299,0.235307,0.128008
3,3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0,1.0,9,1,3,0,0,0,0.0,1_2,0.235307,0.000000,-0.235307
4,4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421834,434367,2021-10-03,632254,57,2021,ATL,NYM,"Gsellman, Robert",663586,607229,Bot,8,1,5,0,5,0,5,5,0.0,0.0,0.0,strikeout,0.000,-0.163,0,5,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141
421835,434368,2021-10-03,632254,58,2021,ATL,NYM,"Gsellman, Robert",594807,607229,Bot,8,2,5,0,5,0,5,5,0.0,0.0,0.0,field_out,0.000,-0.103,0,5,0,0.0,9,1,3,0,0,0,0.0,0_2,0.107299,0.000000,-0.107299
421836,434369,2021-10-03,632254,59,2021,ATL,NYM,"Smith, Will",641645,519293,Top,9,0,5,0,5,0,0,0,0.0,0.0,0.0,strikeout,0.003,-0.242,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
421837,434370,2021-10-03,632254,60,2021,ATL,NYM,"Smith, Will",607043,519293,Top,9,1,5,0,5,0,0,0,0.0,0.0,0.0,field_out,0.002,-0.163,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141


# Export RV data

In [177]:
# data_pa_drop.to_csv('../data/data_RV.csv')

# Prediction

In [49]:
cluster = pd.read_csv('../data/cluster.csv', index_col=[0])

In [50]:
test_df = data_pa_drop[data_pa_drop['game_date']>='2021-09-01'].copy()
train_df = data_pa_drop[data_pa_drop['game_date']<'2021-09-01'].copy()

In [51]:
test_df

,index,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,pre_on_base,max_inning,is_last_out,post_outs,post_1b,post_2b,post_3b,post_on_base,pre_baseOutState,pre_RE,post_RE,run_value
389317,400848,2021-09-01,632640,1,2021,TOR,BAL,"Matz, Steven",656775,571927,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.241,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
389318,400849,2021-09-01,632640,2,2021,TOR,BAL,"Matz, Steven",663624,571927,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.025,0.263,0,0,0,0.0,9,0,1,1,0,0,1.0,0_1,0.280440,0.556846,0.276406
389319,400850,2021-09-01,632640,3,2021,TOR,BAL,"Matz, Steven",669720,571927,Top,1,1,0,0,0,0,0,0,1.0,0.0,0.0,force_out,0.028,-0.384,0,0,0,1.0,9,0,2,1,0,0,1.0,1_1,0.556846,0.235307,-0.321539
389320,400851,2021-09-01,632640,4,2021,TOR,BAL,"Matz, Steven",641820,571927,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.295,0,0,0,1.0,9,1,3,0,0,0,0.0,1_2,0.235307,0.000000,-0.235307
389321,400852,2021-09-01,632640,5,2021,TOR,BAL,"Harvey, Matt",543807,518774,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,-0.021,-0.241,0,1,1,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421834,434367,2021-10-03,632254,57,2021,ATL,NYM,"Gsellman, Robert",663586,607229,Bot,8,1,5,0,5,0,5,5,0.0,0.0,0.0,strikeout,0.000,-0.163,0,5,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141
421835,434368,2021-10-03,632254,58,2021,ATL,NYM,"Gsellman, Robert",594807,607229,Bot,8,2,5,0,5,0,5,5,0.0,0.0,0.0,field_out,0.000,-0.103,0,5,0,0.0,9,1,3,0,0,0,0.0,0_2,0.107299,0.000000,-0.107299
421836,434369,2021-10-03,632254,59,2021,ATL,NYM,"Smith, Will",641645,519293,Top,9,0,5,0,5,0,0,0,0.0,0.0,0.0,strikeout,0.003,-0.242,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861
421837,434370,2021-10-03,632254,60,2021,ATL,NYM,"Smith, Will",607043,519293,Top,9,1,5,0,5,0,0,0,0.0,0.0,0.0,field_out,0.002,-0.163,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141


In [52]:
pitchers_cluster = cluster['pitcher'].to_list()
pitchers_cluster = cluster['pitcher'].to_list()

batter_new = train_df['batter'].value_counts().rename_axis('batter').reset_index(name='n_PA')
batter_new = batter_new[batter_new['n_PA']<100]['batter'].to_list()

In [53]:
test_df['is_newPitcher'] = 0
test_df['is_newBatter'] = 0
for index, row in test_df.iterrows():
    if row.pitcher not in pitchers_cluster:
        test_df.at[index, 'is_newPitcher'] = 1
    elif row.batter in batter_new:
        test_df.at[index, 'is_newBatter'] = 1
    else:
        continue

# past RV search operation

In [54]:
train_df = train_df.merge(cluster, on='pitcher', how='right')

In [55]:
train_df.head()

,index,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name_x,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,pre_on_base,max_inning,is_last_out,post_outs,post_1b,post_2b,post_3b,post_on_base,pre_baseOutState,pre_RE,post_RE,run_value,player_name_y,throws_right,labels_KM,labels_HC,labels_GMM
0,16400,2019-04-13,567463,1,2019,NYY,CWS,"Sabathia, CC",544725,282332,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.249,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861,"Sabathia, CC",0,0,1,1
1,16401,2019-04-13,567463,2,2019,NYY,CWS,"Sabathia, CC",641313,282332,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,0.015,-0.177,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141,"Sabathia, CC",0,0,1,1
2,16402,2019-04-13,567463,3,2019,NYY,CWS,"Sabathia, CC",547989,282332,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.010,-0.115,0,0,0,0.0,9,1,3,0,0,0,0.0,0_2,0.107299,0.000000,-0.107299,"Sabathia, CC",0,0,1,1
3,16408,2019-04-13,567463,9,2019,NYY,CWS,"Sabathia, CC",475174,282332,Top,2,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,0.023,-0.249,0,0,0,0.0,9,0,1,0,0,0,0.0,0_0,0.525301,0.280440,-0.244861,"Sabathia, CC",0,0,1,1
4,16409,2019-04-13,567463,10,2019,NYY,CWS,"Sabathia, CC",660162,282332,Top,2,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.017,-0.177,0,0,0,0.0,9,0,2,0,0,0,0.0,0_1,0.280440,0.107299,-0.173141,"Sabathia, CC",0,0,1,1


In [56]:
RV_KM = train_df.groupby(['batter', 'labels_KM', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()
RV_HC = train_df.groupby(['batter', 'labels_HC', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()
RV_GMM = train_df.groupby(['batter', 'labels_GMM', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()
RV_leagueAVG = train_df.groupby('pre_baseOutState').agg({'run_value':'mean'}).reset_index()

RV_KMavgBatter = train_df.groupby(['labels_KM', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()
RV_HCavgBatter = train_df.groupby(['labels_HC', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()
RV_GMMavgBatter = train_df.groupby(['labels_GMM', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()


RV_avgPitcher = train_df.groupby(['batter', 'pre_baseOutState']).agg({'run_value':'mean'}).reset_index()

In [69]:
test_df[test_df['run_value_pred'].notna()]['run_value'].mean()

-0.04542139517730729

In [70]:
test_df[test_df['run_value_pred'].notna()]['run_value_pred'].mean()

0.0043458220432693335

In [ ]:
count=0
for index, row in test_df.iterrows():
    batter = row.batter
    pitcher = row.pitcher
    state = row.pre_baseOutState
    try:
        if batter in batter_new and pitcher not in pitchers_cluster:
            test_df.at[index, 'run_value_pred'] = RV_leagueAVG[RV_leagueAVG['pre_baseOutState']==state]['run_value']
        elif batter in batter_new and pitcher in pitchers_cluster:
            pitcher_label = int(cluster[cluster['pitcher']==pitcher]['labels_KM'])
            test_df.at[index, 'run_value_pred'] = RV_KMavgBatter[(RV_KMavgBatter['labels_KM']==pitcher_label)&(RV_KMavgBatter['pre_baseOutState']==state)]['run_value']
        elif batter not in batter_new and pitcher not in pitchers_cluster:
            test_df.at[index, 'run_value_pred'] = RV_avgPitcher[(RV_avgPitcher.batter==batter)&(RV_avgPitcher.pre_baseOutState==state)]['run_value']
        else:
            pitcher_label = int(cluster[cluster['pitcher']==pitcher]['labels_KM'])
            test_df.at[index, 'run_value_pred'] = RV_KM[(RV_KM.batter==batter)&(RV_KM.labels_KM==pitcher_label)&(RV_KM.pre_baseOutState==state)]['run_value']
    except:
            test_df.at[index, 'run_value_pred'] = RV_KMavgBatter[(RV_KMavgBatter['labels_KM']==pitcher_label)&(RV_KMavgBatter['pre_baseOutState']==state)]['run_value']
    

In [168]:
count=0
for index, row in test_df.iterrows():
    batter = row.batter
    pitcher = row.pitcher
    state = row.pre_baseOutState
    
    test_df.at[index, 'RV_pred_avg'] = float(RV_leagueAVG[RV_leagueAVG['pre_baseOutState']==state]['run_value'])

In [170]:
test_df['run_value'].mean()

0.007238104446132649

In [175]:
test_df['run_value_pred'].mean()

0.007123436477889168

In [171]:
test_df['RV_pred_avg'].mean()

0.004043946678107215

In [174]:
train_df['labels_KM'].unique()

array([0, 1, 3, 2])